In [3]:
!pip install dash dash_table

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB 435.7 kB/s eta 0:00:18
   ---------------------------------------- 0.0/7.8 MB 435.7 kB/s eta 0:00:18
   ---------------------------------------- 0.0/7.8 MB 219.4 kB/s eta 0:00:36
   ---------------------------------------- 0.1/7.8 MB 353.1 kB/s eta 0:00:22
   ---------------------------------------- 0.1/7.8 MB 350.1 kB/s eta 0:00:22
    --------------------------------------- 0.1/7.8 MB 328.2 kB/s eta 0:00:24
    --------------------------------------- 0.1/7.8 MB 370.8 kB/s eta 0:00:21
   - -------------------------------------- 0.2/7.8 MB 529.7 kB/s eta 0:00:15
   - -------------------------------------- 0.2/7.8 MB 529.7 kB/s eta 0:00:15
   - -------------------------------------- 0.2/7.8 MB 486.6 kB/s eta 0:00:16
   - -------------------------------------- 0.3/7.8 MB 617.8 kB/s eta 0:00:13
   -

In [49]:
import pandas as pd

# 1. 讀取資料：請自行確認CSV欄位是否包含下列所需欄位
CSV_URL = "https://raw.githubusercontent.com/TBNworkGroup/Specieslist_Validation_Dashboard/refs/heads/main/data/output/TT_duplicates_result.csv"
df = pd.read_csv(CSV_URL)

# 假設 df 裡至少有這些欄位：
#  taxonUUID, kingdom, simplifiedScientificName, scientificName,
#  dup_simplifiedGlobal, dup_simplifiedKingdom, dup_scientificKingdom

# 2. 轉成 JSON，確保中文不被轉義
data_json = df.to_json(orient="records", force_ascii=False)

# 3. HTML 模板
html_template = r"""
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Species Duplicates Dashboard</title>
  
  <!-- jQuery & DataTables CDN -->
  <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
  <link rel="stylesheet" type="text/css" 
        href="https://cdn.datatables.net/1.13.4/css/jquery.dataTables.min.css"/>
  <script src="https://cdn.datatables.net/1.13.4/js/jquery.dataTables.min.js"></script>

  <style>
    body { font-family: Arial, sans-serif; margin: 20px; }
    h1 { margin-bottom: 10px; }
    .filter-section { margin: 10px 0; }
    .filter-section label { margin-right: 5px; }
    table.dataTable thead th {
      white-space: nowrap;
    }
  </style>
</head>
<body>
  <h1>Species Duplicates Dashboard</h1>
  
  <div class="filter-section">
    <label for="dup-type-select">重複類型:</label>
    <select id="dup-type-select">
      <!-- 這裡只列出二選一的篩選類型；若需要加選項請自行修改 -->
      <option value="SimplifiedKingdom">學名重複</option>
      <option value="ScientificKingdom">學名+命名者重複</option>
    </select>

    <label for="kingdom-select">篩選 Kingdom:</label>
    <select id="kingdom-select">
      <option value="All" selected>All</option>
      <!-- kingdom 選項會動態填入 -->
    </select>
  </div>

  <!-- 顯示資料表(不包含布林欄位) -->
  <table id="duplicates-table" class="display" style="width:100%">
    <thead>
      <tr>
        <th>taxonUUID</th>
        <th>kingdom</th>
        <th>simplifiedScientificName</th>
        <th>scientificName</th>
        <th>TT_URL</th>
      </tr>
    </thead>
    <tbody>
      <!-- JS 動態生成 -->
    </tbody>
  </table>

  <script>
    // 將 Python 注入的資料放在此變數
    const globalData = __DATA_JSON__;

    // DataTable 物件
    let myDataTable = null;

    //== 主流程：立即執行 ==
    (function main(){
      // 1. 取得 unique kingdom
      populateKingdomDropdown();

      // 2. 初始化表格
      initDataTable(globalData);

      // 3. 監聽下拉選單改變事件
      document.getElementById("dup-type-select")
              .addEventListener("change", updateDataTable);
      document.getElementById("kingdom-select")
              .addEventListener("change", updateDataTable);
    })();

    //== 填入 kingdom 下拉選單 ==
    function populateKingdomDropdown(){
      const kingdoms = globalData.map(d => d.kingdom).filter(k => k);
      const uniqueKingdoms = [...new Set(kingdoms)].sort();

      const kingdomSelect = document.getElementById("kingdom-select");
      // 在預設的 All 之外，動態新增選項
      uniqueKingdoms.forEach(k => {
        const opt = document.createElement("option");
        opt.value = k;
        opt.textContent = k;
        kingdomSelect.appendChild(opt);
      });
    }

    //== 初始化 DataTable ==
    function initDataTable(dataArray){
      fillTableBody(dataArray);
      myDataTable = $('#duplicates-table').DataTable({
        pageLength: 10,
        order: []
      });
    }

    //== 填入 <tbody> (不顯示三個布林欄位) ==
    function fillTableBody(dataArray){
      const tbody = document.querySelector('#duplicates-table tbody');
      tbody.innerHTML = ''; // 先清空

      dataArray.forEach(row => {
        const tr = document.createElement('tr');
        // 只顯示這五個欄位
        const cols = [
          row.taxonUUID,
          row.kingdom,
          row.simplifiedScientificName,
          row.scientificName,
          row.TT_URL
        ];
        cols.forEach(val => {
          const td = document.createElement('td');
          td.textContent = val;
          tr.appendChild(td);
        });
        tbody.appendChild(tr);
      });
    }

    //== 篩選並重繪 DataTables ==
    function updateDataTable(){
      const dupType = document.getElementById("dup-type-select").value;
      const selectedKingdom = document.getElementById("kingdom-select").value;

      // 1. 基本篩選：依 duplication type
      let filtered = [];
      if (dupType === "SimplifiedKingdom") {
        // 只保留 dup_simplifiedKingdom == True (或 'True')
        filtered = globalData.filter(d =>
          d.dup_simplifiedKingdom === true ||
          d.dup_simplifiedKingdom === "True"
        );
      } else if (dupType === "ScientificKingdom") {
        // 只保留 dup_scientificKingdom == True
        filtered = globalData.filter(d =>
          d.dup_scientificKingdom === true ||
          d.dup_scientificKingdom === "True"
        );
      } else {
        // 若還想保留其他選項(例如 "All")，可以自行加邏輯
        filtered = globalData;
      }

      // 2. 如果 kingdom != 'All'，再依 kingdom 過濾
      if (selectedKingdom !== "All") {
        filtered = filtered.filter(d => d.kingdom === selectedKingdom);
      }

      // 3. 重啟 DataTable
      if (myDataTable) {
        myDataTable.clear().destroy();
      }
      fillTableBody(filtered);
      myDataTable = $('#duplicates-table').DataTable({
        pageLength: 10,
        order: []
      });
    }
  </script>
</body>
</html>
"""

# 4. 以 data_json 取代模板裡的 __DATA_JSON__
html_output = html_template.replace("__DATA_JSON__", data_json)

# 5. 指定輸出路徑到 docs 資料夾
output_file = r"C:\Users\jared\Documents\GitHub\Specieslist_Validation_Dashboard\docs\dashboard.html"

with open(output_file, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"已產生 {output_file}，請用瀏覽器打開查看。")

已產生 C:\Users\jared\Documents\GitHub\Specieslist_Validation_Dashboard\docs\dashboard.html，請用瀏覽器打開查看。


In [41]:
import pandas as pd

# 1. 讀取資料：請自行確認CSV欄位是否包含下列所需欄位
CSV_URL = "https://raw.githubusercontent.com/TBNworkGroup/Specieslist_Validation_Dashboard/refs/heads/main/data/output/TT_duplicates_result.csv"
df = pd.read_csv(CSV_URL)
df

,taxonUUID,taxonRank,kingdom,simplifiedScientificName,scientificName,dup_simplifiedGlobal,dup_simplifiedKingdom,dup_scientificKingdom,TT_URL
0,c81c2d9d-6ab5-461f-ba33-48c46fd7a98f,domain,NaN,Bacteria,Bacteria,True,False,False,https://taxatree.tbn.org.tw/taxa/c81c2d9d-6ab5...
1,04a87524-3ed5-495d-ab3e-a5689ba55487,order,Animalia,Pholidota,Pholidota,True,False,False,https://taxatree.tbn.org.tw/taxa/04a87524-3ed5...
2,fe0b9853-61fa-47f3-8850-e8aa646cb9cb,order,Animalia,Heteropoda,Heteropoda,True,True,True,https://taxatree.tbn.org.tw/taxa/fe0b9853-61fa...
3,856a1a16-e8ac-436a-9ad2-ac5aff334344,family,Animalia,Centriscidae,Centriscidae,True,True,True,https://taxatree.tbn.org.tw/taxa/856a1a16-e8ac...
4,ed03b9bc-c719-4bb9-8add-e9657bfbf7b1,family,Animalia,Centriscidae,Centriscidae,True,True,True,https://taxatree.tbn.org.tw/taxa/ed03b9bc-c719...
...,...,...,...,...,...,...,...,...,...
171,b152bcf1-afc5-4a80-8125-2de3eebd3e8c,species,Plantae,Kurzia pauciflora,Kurzia pauciflora (Dicks.) Grolle,True,True,False,https://taxatree.tbn.org.tw/taxa/b152bcf1-afc5...
172,f609f637-f160-40be-bd6b-1e7e78050833,family,Fungi,Sympoventuriaceae,Sympoventuriaceae,True,True,True,https://taxatree.tbn.org.tw/taxa/f609f637-f160...
173,f28abec8-e3e7-4da9-8ee5-b869a445d597,genus,Animalia,Manulea,Manulea,True,False,False,https://taxatree.tbn.org.tw/taxa/f28abec8-e3e7...
174,2892f3ca-35f9-4b07-b812-87241d263efb,genus,Plantae,Anisophyllum,Anisophyllum,True,True,True,https://taxatree.tbn.org.tw/taxa/2892f3ca-35f9...
